# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />   

### Tarea 2: Regresión y clasificación lineal

Un club del juego de Go recopiló los resultados de varias partidas entre diferentes jugadores, almacenados en el archivo partidas_entrenamiento.txt, con el objetivo de predecir el resultado de
partidas futuras, ejemplos de las cuales se encuentran en el archivo partidas_prueba.txt. Los archivos partidas_entrenamiento.txt y partidas_prueba.txt contienen 3 columnas: la primera
corresponde al identificador del jugador A, la segunda al identificador del jugador B y la tercera es
el resultado de la partida (1 si ganó el jugador A o 0 si ganó el jugador B). En el club hay un total
de D jugadores, por lo que cada identificador es un número entero entre 1 y D. La predicción del
resultado de un juego se puede plantear como un problema de clasificación: dados 2 jugadores (A y
B) se requiere predecir si A ganó (y = 1) o si fue B (y = 0).

In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score, roc_curve, roc_auc_score
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import accuracy_score

import plotly.express as px
import plotly.graph_objects as go

from tqdm import tqdm
import numpy as np

import sys
import os

### A

Entrena y evalúa un clasificador bayesiano ingenuo. Al ser un modelo generativo (modela
la probabilidad conjunta P (x, y)), es posible generar partidas artificiales con los parámetros
calculados. Genera nuevas partidas que sigan la distribución modelada

In [2]:
# Read juegos_entrenamiento.txt juegos_validacion.txt

df_train = pd.read_csv('regl_data/juegos_entrenamiento.txt', sep=' ')
df_train.columns = ['A', 'B', 'Win']
df_valid = pd.read_csv('regl_data/juegos_validacion.txt', sep=' ')
df_valid.columns = ['A', 'B', 'Win']

In [3]:
df_train.describe()

,A,B,Win
count,318.000000,318.000000,318.000000
mean,61.827044,64.226415,0.522013
std,42.756795,43.591656,0.500302
min,1.000000,1.000000,0.000000
25%,26.000000,23.000000,0.000000
50%,55.000000,55.500000,1.000000
75%,97.000000,105.000000,1.000000
max,142.000000,141.000000,1.000000


In [4]:
df_train

,A,B,Win
0,7,8,1
1,40,10,1
2,16,17,1
3,8,18,1
4,112,40,0
...,...,...,...
313,80,97,1
314,81,45,0
315,35,43,1
316,87,23,0


In [5]:
print(df_train.max())
print(df_train.min())
print(df_valid.max())
print(df_valid.min())

# Interesante...

A      142
B      141
Win      1
dtype: int64
A      1
B      1
Win    0
dtype: int64
A      132
B      136
Win      1
dtype: int64
A      4
B      4
Win    0
dtype: int64


Visualizamos los datos de entrenamiento. (A vs B)

In [6]:
# Plot A vs B
fig = px.scatter(df_train, x='A', y='B', color='Win')
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='A vs B', xaxis_title='A', yaxis_title='B')
fig.show()

In [7]:
#Entrena y evalúa un clasificador bayesiano ingenuo. Al ser un modelo generativo (modela
#la probabilidad conjunta P (x, y)), es posible generar partidas artificiales con los parámetros
#calculados. Genera nuevas partidas que sigan la distribución modelada.

from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

X_train = df_train[['A', 'B']].to_numpy()
y_train = df_train['Win'].to_numpy()
X_valid = df_valid[['A', 'B']].to_numpy()
y_valid = df_valid['Win'].to_numpy()

model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_valid)
y_pred = model.predict(X_valid)

In [8]:
fpr, tpr, thresholds = roc_curve(y_valid,  y_pred)
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_proba[:, 1])
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')

auc = roc_auc_score(y_valid, y_pred)
print(f'AUC: {auc}')
auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
print(f'AUC: {auc}')

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guessing', line=dict(dash='dash')))
fig.update_layout(title='ROC Curve',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate')
fig.show()

FPR: [0.         0.78787879 1.        ]
TPR: [0.         0.86419753 1.        ]
FPR: [0.         0.         0.         0.06060606 0.06060606 0.06060606
 0.09090909 0.09090909 0.12121212 0.15151515 0.15151515 0.15151515
 0.18181818 0.18181818 0.24242424 0.24242424 0.27272727 0.27272727
 0.27272727 0.33333333 0.45454545 0.45454545 0.57575758 0.66666667
 0.66666667 0.6969697  0.6969697  0.72727273 0.72727273 0.78787879
 0.81818182 0.81818182 0.90909091 0.93939394 0.93939394 0.93939394
 0.93939394 0.96969697 1.         1.        ]
TPR: [0.         0.01234568 0.03703704 0.03703704 0.0617284  0.08641975
 0.08641975 0.11111111 0.19753086 0.19753086 0.27160494 0.28395062
 0.2962963  0.32098765 0.35802469 0.37037037 0.37037037 0.38271605
 0.40740741 0.40740741 0.41975309 0.44444444 0.66666667 0.74074074
 0.79012346 0.80246914 0.81481481 0.81481481 0.82716049 0.86419753
 0.86419753 0.87654321 0.91358025 0.92592593 0.9382716  0.96296296
 0.97530864 0.98765432 0.98765432 1.        ]
AUC: 0.5381593

FPR y TPR muestran un comportamiento típico de un modelo con cierto nivel de discriminación.

AUC: 0.5381 (para predicciones binarias) y 0.5574 (para probabilidades). 
Estos valores indican que el modelo tiene un desempeño apenas mejor que un clasificador aleatorio.

Esto quiere decir que el modelo no está capturando bien la relación entre las características y las etiquetas.

##### Generar Partidas Artificiales

Recordemos que a probabilidad conjunta es una medida en probabilidad que describe la probabilidad de que dos o más eventos ocurran simultáneamente. 

En NaiveBayes, la probabilidad conjunta se refiere a la probabilidad de observar un conjunto de características ( X ) y una clase ( y ), es decir, ( $P(X, y)$ ).

$$[ P(X, y) = P(y) \cdot P(X \mid y) ]$$

Donde:

( P(y) ): Es la probabilidad a priori de la clase ( y ).
( $P(X \mid y)$ ): Es la probabilidad condicional de las características ( X ) dado ( y ). En el caso de Gaussian Naive Bayes, se asume que cada característica sigue una distribución normal (gaussiana) y que las características son independientes entre sí.
El modelo calcula ( P(X, y) ) para cada clase ( y ) y luego utiliza la regla de Bayes para predecir la clase más probable ( y ) para un conjunto de características ( X ):

$$[ P(y \mid X) = \frac{P(X, y)}{P(X)} ]$$

Dado que ( P(X) ) es constante para todas las clases, el modelo simplemente maximiza ( P(X, y) ) para determinar la clase más probable.



In [14]:
# Importar el modelo GaussianNaiveBayes de la Tarea 1
sys.path.append(os.path.abspath('../T1'))
from CustomNB import GaussianNaiveBayes

model = GaussianNaiveBayes(convariance=True)

model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
accuracy = np.mean(y_pred == y_valid)

print(f'Accuracy: {accuracy:.2f}')

def generate_artificial_games(model, n_games=1000):
    """
    Generate artificial games using the trained model.
    """

    means = model.mean  # Array 2D: (n_classes, n_features)
    covariances = model.covariances  # Array 3D: (n_classes, n_features, n_features)
    class_priors = model.prior  # Probabilidades a priori de cada clase

    n_classes = len(model.classes)
    n_features = model.features
    samples = np.zeros((n_games, n_features))
    labels = np.zeros(n_games)

    for i in range(n_games):
        class_idx = np.random.choice(n_classes, p=class_priors)
        sample = np.random.multivariate_normal(means[class_idx], covariances[class_idx])
        sample = np.clip(sample, X_train.min(), X_train.max())
        #sample = np.maximum(np.random.multivariate_normal(means[class_idx], covariances[class_idx]), 0)
        samples[i] = sample
        labels[i] = class_idx

    samples_df = pd.DataFrame(samples, columns=['A', 'B']).astype(int)
    labels_df = pd.DataFrame(labels, columns=['Win'])
    samples_df = pd.concat([samples_df, labels_df], axis=1)
    samples_df.columns = ['A', 'B', 'Win']
    samples_df['Win'] = samples_df['Win'].astype(int)

    return samples_df

# Generate 1000 artificial games
artificial_games = generate_artificial_games(model, n_games=1000)

fig = px.scatter(artificial_games, x='A', y='B', color='Win',
                 title='Artificial Games', labels={'A': 'A', 'B': 'B', 'Win': 'Win'})
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Artificial Games',
                  xaxis_title='A',
                  yaxis_title='B')
fig.show()

Accuracy: 0.68


In [16]:
# Si queremos generar partidas artificiales tambien podemos la función predict_proba de scikit-learn

X_train = df_train[['A', 'B']]
y_train = df_train['Win']
X_valid = df_valid[['A', 'B']]
y_valid = df_valid['Win']

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
y_pred_proba = gnb.predict_proba(X_valid)

def generar_partida(A, B):
    X_input = pd.DataFrame([[A, B]], columns=['A', 'B'])
    prob_A = gnb.predict_proba(X_input)[0][1]
    prob_B = 1 - prob_A
    resultado = np.random.choice([1, 0], p=[prob_A, prob_B])
    return resultado

partidas_artificiales = []
for _ in range(10):
    jugadores = X_train[['A', 'B']].sample(2)
    jugador_A = jugadores.iloc[0]['A']
    jugador_B = jugadores.iloc[1]['B']
    resultado = generar_partida(jugador_A, jugador_B)
    partidas_artificiales.append((jugador_A, jugador_B, resultado))

print("Partidas artificiales generadas:")
for partida in partidas_artificiales:
    print(f"Jugador A: {partida[0]}, Jugador B: {partida[1]}, Resultado: {partida[2]}")

Partidas artificiales generadas:
Jugador A: 94, Jugador B: 101, Resultado: 1
Jugador A: 4, Jugador B: 105, Resultado: 1
Jugador A: 53, Jugador B: 18, Resultado: 0
Jugador A: 27, Jugador B: 18, Resultado: 1
Jugador A: 7, Jugador B: 11, Resultado: 1
Jugador A: 4, Jugador B: 59, Resultado: 0
Jugador A: 69, Jugador B: 9, Resultado: 0
Jugador A: 30, Jugador B: 57, Resultado: 0
Jugador A: 107, Jugador B: 136, Resultado: 0
Jugador A: 8, Jugador B: 18, Resultado: 1


In [ ]:
# Generar partidas artificiales
partidas_artificiales = []
for _ in range(1000):
    jugadores = X_train[['A', 'B']].sample(2)
    jugador_A = jugadores.iloc[0]['A']
    jugador_B = jugadores.iloc[1]['B']
    resultado = generar_partida(jugador_A, jugador_B)
    partidas_artificiales.append((jugador_A, jugador_B, resultado))

partidas_df = pd.DataFrame(partidas_artificiales, columns=['A', 'B', 'Win'])

fig = px.scatter(partidas_df, x='A', y='B', color='Win',
                 title='Artificial Games', labels={'A': 'A', 'B': 'B', 'Win': 'Win'})
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Artificial Games',
                  xaxis_title='A',
                  yaxis_title='B')
fig.show()

## B

Entrena y evalúa un clasificador de regresión logística. Debido a que ambos atributos son
categóricos, es necesario cambiar la codificación. Explica el procedimiento y la lógica de la
codificación que realizaste. Visualiza los valores de los parámetros del modelo de regresión
logística y discute qué interpretación tendrían de acuerdo a la codificación realizada. Grafica
las curvas ROC y de precisión-exhaustividad y reporta sus áreas bajo la curva.

In [18]:
from CustomLR import BinaryLogisticRegression

X_train = df_train[['A', 'B']].to_numpy()
y_train = df_train['Win'].to_numpy()
X_valid = df_valid[['A', 'B']].to_numpy()
y_valid = df_valid['Win'].to_numpy()

model = BinaryLogisticRegression(lr=0.01, epochs=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
accuracy = np.mean(y_pred == y_valid)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.32


In [23]:
model.weights

array([ 0.03018595, -0.31737639])

In [25]:
# Visualiza los valores de los parámetros del modelo de regresión logística

# array([ 0.03018595, -0.31737639])

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[model.weights[0], model.weights[1]], mode='markers', name='Logistic Regression'))
fig.update_layout(title='Logistic Regression Parameters',
                  xaxis_title='A',
                  yaxis_title='B')
fig.show()

In [13]:
fpr, tpr, thresholds = roc_curve(y_valid,  y_pred)
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_proba[:, 1])
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')

auc = roc_auc_score(y_valid, y_pred)
print(f'AUC: {auc}')
auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
print(f'AUC: {auc}')

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guessing', line=dict(dash='dash')))
fig.update_layout(title='ROC Curve',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate')
fig.show()

FPR: [0. 0. 1.]
TPR: [0.         0.04938272 1.        ]
FPR: [0.         0.         0.         0.06060606 0.06060606 0.06060606
 0.09090909 0.09090909 0.12121212 0.15151515 0.15151515 0.15151515
 0.18181818 0.18181818 0.24242424 0.24242424 0.27272727 0.27272727
 0.27272727 0.33333333 0.45454545 0.45454545 0.57575758 0.66666667
 0.66666667 0.6969697  0.6969697  0.72727273 0.72727273 0.78787879
 0.81818182 0.81818182 0.90909091 0.93939394 0.93939394 0.93939394
 0.93939394 0.96969697 1.         1.        ]
TPR: [0.         0.01234568 0.03703704 0.03703704 0.0617284  0.08641975
 0.08641975 0.11111111 0.19753086 0.19753086 0.27160494 0.28395062
 0.2962963  0.32098765 0.35802469 0.37037037 0.37037037 0.38271605
 0.40740741 0.40740741 0.41975309 0.44444444 0.66666667 0.74074074
 0.79012346 0.80246914 0.81481481 0.81481481 0.82716049 0.86419753
 0.86419753 0.87654321 0.91358025 0.92592593 0.9382716  0.96296296
 0.97530864 0.98765432 0.98765432 1.        ]
AUC: 0.5246913580246914
AUC: 0.5574261

In [ ]:
# Ahora con Sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

X_train = df_train[['A', 'B']]
y_train = df_train['Win']
X_valid = df_valid[['A', 'B']]
y_valid = df_valid['Win']

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_proba = model.predict_proba(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(f'Weights: {model.coef_}')
print(f'Intercept: {model.intercept_}')

# Visualiza los valores de los parámetros del modelo de regresión logística

fig = go.Figure()

# Agregar los pesos al gráfico
fig.add_trace(go.Scatter(
    x=list(range(len(model.coef_[0]))),  # Índices de los pesos
    y=model.coef_[0],
    mode='markers',
    name='Pesos'
))

# Agregar el intercepto al gráfico
#fig.add_trace(go.Scatter(
#    x=[-1],  # Índice para el intercepto
#    y=[model.intercept_[0]],  # Valor del intercepto
#    mode='markers',
#    name='Intercepto'
#))

# Configurar el diseño del gráfico
fig.update_layout(
    title='Pesos e Intercepto del Modelo de Regresión Logística',
    xaxis_title='Índice',
    yaxis_title='Valor',
    showlegend=True
)

# Mostrar el gráfico
fig.show()

fpr, tpr, thresholds = roc_curve(y_valid,  y_pred)
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_proba[:, 1])
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')
auc = roc_auc_score(y_valid, y_pred)
print(f'AUC: {auc}')
auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
print(f'AUC: {auc}')

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guessing', line=dict(dash='dash')))
fig.update_layout(title='ROC Curve',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate')
fig.show()

Accuracy: 0.68
Weights: [[ 0.0035675  -0.00546468]]
Intercept: [0.21974074]


FPR: [0.         0.75757576 1.        ]
TPR: [0.         0.85185185 1.        ]
FPR: [0.         0.         0.         0.06060606 0.06060606 0.06060606
 0.06060606 0.09090909 0.09090909 0.09090909 0.12121212 0.12121212
 0.15151515 0.18181818 0.18181818 0.18181818 0.21212121 0.33333333
 0.36363636 0.36363636 0.42424242 0.42424242 0.45454545 0.45454545
 0.57575758 0.66666667 0.6969697  0.75757576 0.75757576 0.78787879
 0.78787879 0.81818182 0.90909091 0.93939394 0.93939394 0.96969697
 0.96969697 1.         1.        ]
TPR: [0.         0.01234568 0.02469136 0.02469136 0.04938272 0.07407407
 0.09876543 0.18518519 0.25925926 0.27160494 0.27160494 0.28395062
 0.2962963  0.2962963  0.30864198 0.35802469 0.35802469 0.37037037
 0.37037037 0.38271605 0.41975309 0.44444444 0.44444444 0.45679012
 0.67901235 0.75308642 0.7654321  0.80246914 0.85185185 0.85185185
 0.87654321 0.87654321 0.91358025 0.92592593 0.9382716  0.95061728
 0.97530864 0.97530864 1.        ]
AUC: 0.5471380471380471
AUC: 0.56004

In [41]:
# Ahora con Sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

X_train = df_train[['A', 'B']].to_numpy()
y_train = df_train['Win'].to_numpy()
X_valid = df_valid[['A', 'B']].to_numpy()
y_valid = df_valid['Win'].to_numpy()

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)  # Ajusta y transforma los datos de entrenamiento
X_valid_encoded = encoder.transform(X_valid) 

model = LogisticRegression()
model.fit(X_train_encoded, y_train)
y_pred = model.predict(X_valid_encoded)

y_pred_proba = model.predict_proba(X_valid_encoded)
accuracy = accuracy_score(y_valid, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(f'Weights: {model.coef_}')
print(f'Intercept: {model.intercept_}')

# Visualiza los valores de los parámetros del modelo de regresión logística

fig = go.Figure()

# Agregar los pesos al gráfico
fig.add_trace(go.Scatter(
    x=list(range(len(model.coef_[0]))),  # Índices de los pesos
    y=model.coef_[0],
    mode='markers',
    name='Pesos'
))

# Agregar el intercepto al gráfico
#fig.add_trace(go.Scatter(
#    x=[-1],  # Índice para el intercepto
#    y=[model.intercept_[0]],  # Valor del intercepto
#    mode='markers',
#    name='Intercepto'
#))

# Configurar el diseño del gráfico
fig.update_layout(
    title='Pesos e Intercepto del Modelo de Regresión Logística',
    xaxis_title='Índice',
    yaxis_title='Valor',
    showlegend=True
)

# Mostrar el gráfico
fig.show()

fpr, tpr, thresholds = roc_curve(y_valid,  y_pred)
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_proba[:, 1])
print(f'FPR: {fpr}')
print(f'TPR: {tpr}')
auc = roc_auc_score(y_valid, y_pred)
print(f'AUC: {auc}')
auc = roc_auc_score(y_valid, y_pred_proba[:, 1])
print(f'AUC: {auc}')

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guessing', line=dict(dash='dash')))
fig.update_layout(title='ROC Curve',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate')
fig.show()

Accuracy: 0.70
Weights: [[ 0.70314479 -0.28560252  0.34434111 -1.24119799  0.2728649  -0.91212754
  -0.41060691  0.21151567  0.31399073 -0.33138142  0.3148287  -0.15363917
  -0.20934593  0.49312579 -0.75176316  0.75927079 -0.13027715  0.34233434
   0.75856896 -0.58801095 -0.2925634   0.83273192 -0.10053665  0.03494281
  -0.29594265  0.9933949  -0.38703643 -0.33424474 -0.47599352 -0.27539119
   0.07632943 -1.11289687  0.59956243  0.29105252  0.34233434  0.33204429
  -0.78491579 -0.15043426 -0.16021561  0.28339207  0.59455011 -0.27010457
  -0.30884635 -0.54630903  0.17772631  0.39514099 -0.55994165  0.0564739
  -0.07910603 -0.29594265 -0.29117317  0.08998972  0.02130273  0.3662201
  -0.33424474  0.3620887  -0.1010642   0.34434111  0.70072175  0.42191944
  -0.61393433 -0.19281701  0.4036082   0.19199321 -0.28667697 -0.12306909
  -0.36690867  0.0296911   0.39288283  0.34233434  0.4486024   0.35972121
  -0.75187916 -0.54630903 -0.36808979  0.80379873  0.34233434 -0.54630903
  -0.63293753 -0

FPR: [0.         0.57575758 1.        ]
TPR: [0.         0.81481481 1.        ]
FPR: [0.         0.03030303 0.03030303 0.03030303 0.03030303 0.15151515
 0.24242424 0.24242424 0.24242424 0.24242424 0.24242424 0.27272727
 0.33333333 0.36363636 0.36363636 0.39393939 0.39393939 0.42424242
 0.42424242 0.45454545 0.54545455 0.54545455 0.57575758 0.57575758
 0.63636364 0.66666667 0.66666667 0.72727273 0.78787879 0.78787879
 0.81818182 0.84848485 0.84848485 0.87878788 1.        ]
TPR: [0.         0.01234568 0.0617284  0.13580247 0.16049383 0.38271605
 0.45679012 0.48148148 0.49382716 0.54320988 0.56790123 0.65432099
 0.69135802 0.69135802 0.7037037  0.7037037  0.74074074 0.74074074
 0.75308642 0.75308642 0.79012346 0.80246914 0.81481481 0.86419753
 0.90123457 0.90123457 0.91358025 0.9382716  0.9382716  0.96296296
 0.96296296 0.97530864 0.98765432 0.98765432 1.        ]
AUC: 0.6195286195286195
AUC: 0.7154882154882155


Observamos que la transformacion de los datos en un formato One-Hot ayuda a mejorar el desempeño del modelo. Esto es porque la regresión logística puede capturar mejor las relaciones no lineales entre las características y la variable objetivo.

En este caso el AUC de la regresión logística es de 0.71, lo que indica que el modelo tiene un desempeño significativamente mejor que el clasificador bayesiano ingenuo.

## C

Compara el clasificador bayesiano ingenuo y regresión logística en este problema. 

¿Qué ventajas y desventajas tienen los modelos entrenados? 

Ambos modelos tienen un desempeño similar en términos de AUC.

El modelo de regresion logistica tiene ventajas en cuanto a la interpretabilidad de los coeficientes, lo que permite entender mejor la relación entre las características y la variable objetivo. No obstante una desventaja es que la regresión logística asume una relación lineal entre las características y la variable objetivo, lo que puede no ser adecuado en todos los casos.

El clasificador bayesiano ingenuo, por otro lado, es más flexible en términos de la forma de la distribución de las características, pero puede ser menos interpretable debido a su naturaleza probabilística. Además, el clasificador bayesiano ingenuo asume independencia entre las características, lo que puede no ser cierto en todos los casos.

¿Qué pasaría si se entrena el clasificador bayesiano ingenuo con los vectores recodificados o si se entrena un modelo de regresión logística usando los vectores de entrada originales?

Si el clasificador bayesiano se entrena con los vectores recodificados, es probable que el modelo no capture adecuadamente la relación entre las características y la variable objetivo, ya que la recodificación puede perder información importante sobre la relación entre las características. 

Por otro lado, como obseramos en el experimento anterior, al utilizar One-Hot Encoding, la regresión logística puede capturar mejor las relaciones no lineales entre las características y la variable objetivo.

¿Consideras que las presuposiciones de cada clasificador son apropiadas para los datos del problema? ¿Para este tipo de problemas cuál de los dos recomendarías y por qué?

Las presuposiciones de cada clasificador son importantes a considerar al elegir el modelo adecuado para un problema específico. En este caso, la regresión logística asume una relación lineal entre las características y la variable objetivo, lo que puede no ser adecuado si los datos tienen una relación no lineal. Por otro lado, el clasificador bayesiano ingenuo asume independencia entre las características, lo que puede no ser cierto en todos los casos.

Para el problema de predecir estas partidas entre jugadores recomendaria utilizar la regresión logística, ya que al menos podemos ver que esta sucediendo con los coeficientes y como se comportan los jugadores entre si, lo cual si no es muy bueno ya es bastante interesante y nos da una idea de que esta sucediendo. 

### D

$$\hat{y} = \text{sigm}(\theta^T x) = \frac{1}{1 + e^{-\theta^T x}}$$
 y 
$$E(\theta) = \frac{1}{2} \sum_{i=1}^{n} \left( \hat{y}^{(i)} - y^{(i)} \right)^2$$